<a href="https://colab.research.google.com/github/Pathway2008/sub_pred/blob/main/sub_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
info = pd.read_csv('/content/drive/MyDrive/predict_subcribe/data_info.csv')
sub = pd.read_csv('/content/drive/MyDrive/predict_subcribe/sample_submission.csv')
train = pd.read_csv('/content/drive/MyDrive/predict_subcribe/train.csv')
test = pd.read_csv('/content/drive/MyDrive/predict_subcribe/test.csv')

In [3]:
train = train.drop(['user_id'],axis=1)

In [4]:
train = train.drop(['target'],axis=1)

In [5]:
test = test.drop(['user_id'],axis=1)

In [22]:
from sklearn.preprocessing import LabelEncoder
labels = [col for col in train.columns if train[col].dtype == 'object' and col not in ['user_id']]
label_encoders = {}
for col in labels:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

train["is_train"] = 0
test["is_train"]  = 1

In [8]:
df = pd.concat([train, test], axis = 0)
df = df.sample(frac = 1, random_state = 123)

In [9]:
X = df.drop(['is_train'], axis = 1)
y = df['is_train']

In [10]:
import pandas as pd
from xgboost import XGBClassifier

In [73]:
xgb_model = XGBClassifier(random_state=42, max_depth=2, scale_pos_weight=sum(y==0)/sum(y==1))
xgb_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [74]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y, xgb_model.predict_proba(X)[:,1])
print(auc_score)

0.646921505


In [75]:
feature_imp_xgb = pd.DataFrame({
    'Feature': list(X.columns),
    'XGB_Score': xgb_model.feature_importances_
})

In [76]:
feature_imp_xgb = feature_imp_xgb.sort_values(by='XGB_Score', ascending=False)
feature_imp_xgb

,Feature,XGB_Score
9,preferred_difficulty_level,0.473978
10,subscription_type,0.259503
1,recent_login_time,0.033135
5,total_completed_courses,0.032388
7,abandoned_learning_sessions,0.031885
0,subscription_duration,0.029616
3,average_time_per_learning_session,0.028920
2,average_login_time,0.022149
12,payment_pattern,0.020712
4,monthly_active_learning_days,0.020666


In [11]:
train.drop(['preferred_difficulty_level','subscription_type'],axis=1,inplace=True)
test.drop(['preferred_difficulty_level','subscription_type'],axis=1,inplace=True)

In [12]:
train.drop(['community_engagement_level'],axis=1,inplace=True)
test.drop(['community_engagement_level'],axis=1,inplace=True)

In [13]:
df = pd.concat([train, test], axis = 0)
df = df.sample(frac = 1, random_state = 123)

X = df.drop(['is_train'], axis = 1)
y = df['is_train']

In [14]:
import pandas as pd
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=42, max_depth=2, scale_pos_weight=sum(y==0)/sum(y==1))
xgb_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [15]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y, xgb_model.predict_proba(X)[:,1])
print(auc_score)

0.59958738


In [82]:
feature_imp_xgb = pd.DataFrame({
    'Feature': list(X.columns),
    'XGB_Score': xgb_model.feature_importances_
})

feature_imp_xgb = feature_imp_xgb.sort_values(by='XGB_Score', ascending=False)
feature_imp_xgb

,Feature,XGB_Score
7,abandoned_learning_sessions,0.142546
4,monthly_active_learning_days,0.131275
1,recent_login_time,0.114044
5,total_completed_courses,0.109809
0,subscription_duration,0.105182
3,average_time_per_learning_session,0.096226
2,average_login_time,0.088168
6,recent_learning_achievement,0.078415
8,customer_inquiry_history,0.070863
9,payment_pattern,0.063471


In [16]:
X_new = X.copy()
X_new['proba'] = xgb_model.predict_proba(X)[:,1]
X_new['target'] = y

In [17]:
X_new = X_new[X_new['target']==0]

In [18]:
nrows = X_new.shape[0]
adversarial_validation_data = X_new.sort_values(by='proba',ascending=False)[:int(nrows*.3)]
adversarial_training_data = X_new.sort_values(by='proba',ascending=False)[int(nrows*.3):]

In [19]:
adversarial_training_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 5800 to 5803
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              7000 non-null   int64  
 1   recent_login_time                  7000 non-null   int64  
 2   average_login_time                 7000 non-null   float64
 3   average_time_per_learning_session  7000 non-null   float64
 4   monthly_active_learning_days       7000 non-null   int64  
 5   total_completed_courses            7000 non-null   int64  
 6   recent_learning_achievement        7000 non-null   float64
 7   abandoned_learning_sessions        7000 non-null   int64  
 8   customer_inquiry_history           7000 non-null   int64  
 9   payment_pattern                    7000 non-null   int64  
 10  proba                              7000 non-null   float32
 11  target                             7000 non-null   int64  

In [20]:
current_index_order = adversarial_training_data.index.tolist()
valid_index = adversarial_validation_data.index.tolist()

In [23]:
x_train = train.loc[current_index_order].reset_index(drop=True)
x_valid = train.loc[valid_index].reset_index(drop=True)

In [24]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            7000 non-null   object 
 1   subscription_duration              7000 non-null   int64  
 2   recent_login_time                  7000 non-null   int64  
 3   average_login_time                 7000 non-null   float64
 4   average_time_per_learning_session  7000 non-null   float64
 5   monthly_active_learning_days       7000 non-null   int64  
 6   total_completed_courses            7000 non-null   int64  
 7   recent_learning_achievement        7000 non-null   float64
 8   abandoned_learning_sessions        7000 non-null   int64  
 9   community_engagement_level         7000 non-null   int64  
 10  preferred_difficulty_level         7000 non-null   int64  
 11  subscription_type                  7000 non-null   int64

In [1]:
pip install autogluon

In [93]:
x_train.to_csv('xgb_train.csv',index=False)
x_valid.to_csv('xgb_valid.csv',index=False)

In [62]:
x_train = pd.read_csv('/content/xgb_train.csv')
x_valid = pd.read_csv('/content/xgb_valid.csv')

In [25]:
x_train['harder'] = 0
x_train.loc[(x_train['average_time_per_learning_session'] > 167), 'harder'] = 1

x_valid['harder'] = 0
x_valid.loc[(x_valid['average_time_per_learning_session'] > 167), 'harder'] = 1

test['harder'] = 0
test.loc[(test['average_time_per_learning_session'] > 167), 'harder'] = 1

In [26]:
x_train['degree'] = x_train['monthly_active_learning_days'] * x_train['average_time_per_learning_session']
x_train['concentrate'] = x_train['average_time_per_learning_session'] - x_train['average_login_time']

x_valid['degree'] = x_valid['monthly_active_learning_days'] * x_valid['average_time_per_learning_session']
x_valid['concentrate'] = x_valid['average_time_per_learning_session'] - x_valid['average_login_time']

test['degree'] = test['monthly_active_learning_days'] * test['average_time_per_learning_session']
test['concentrate'] = test['average_time_per_learning_session'] - test['average_login_time']

In [27]:
x_train.drop(['user_id'],axis=1,inplace=True)
x_valid.drop(['user_id'],axis=1,inplace=True)

In [28]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              7000 non-null   int64  
 1   recent_login_time                  7000 non-null   int64  
 2   average_login_time                 7000 non-null   float64
 3   average_time_per_learning_session  7000 non-null   float64
 4   monthly_active_learning_days       7000 non-null   int64  
 5   total_completed_courses            7000 non-null   int64  
 6   recent_learning_achievement        7000 non-null   float64
 7   abandoned_learning_sessions        7000 non-null   int64  
 8   community_engagement_level         7000 non-null   int64  
 9   preferred_difficulty_level         7000 non-null   int64  
 10  subscription_type                  7000 non-null   int64  
 11  customer_inquiry_history           7000 non-null   int64

In [29]:
from autogluon.tabular import TabularDataset, TabularPredictor
train = TabularDataset(x_train)
valid = TabularDataset(x_valid)
test = TabularDataset(test)

In [30]:
fit_parameters = {
    'presets': 'best_quality',
    'num_bag_folds': 7,
    'num_bag_sets': 5,
    'num_stack_levels': 2
}

In [31]:
predictor = TabularPredictor(
    label='target',
    eval_metric='f1_macro',
    sample_weight='balance_weight'
)

predictor.fit(
    train_data=train,
    tuning_data=valid,
    auto_stack=True,
    fit_weighted_ensemble=True,
    use_bag_holdout=True,
    presets=fit_parameters,
    num_gpus=1,
    hyperparameter_tune_kwargs='auto'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231209_114616"
Presets specified: [{'presets': 'best_quality', 'num_bag_folds': 7, 'num_bag_sets': 5, 'num_stack_levels': 2}]
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=7, num_bag_sets=5
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231209_114616"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 30 11:19:59 UTC 2023
CPU Count:          2
Memory Avail:       11.28 GB / 12.68 GB (89.0%)
Disk Space Avail:   132.63 GB / 166.77 GB (79.5%)
Train Data Rows:    7000
Train Data Columns: 16
Tuning Data Rows:    3000
Tuning Data Columns: 16
Label Colu

  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.07%)
Fitted model: LightGBMXT_BAG_L1/T1 ...
	0.5056	 = Validation score   (f1_macro)
	217.14s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: LightGBM_BAG_L1 ...


  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.08%)
Fitted model: LightGBM_BAG_L1/T1 ...
	0.5218	 = Validation score   (f1_macro)
	226.18s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: RandomForestGini_BAG_L1 ...
	No hyperparameter search space specified for RandomForestGini_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForestGini_BAG_L1 ...
	0.4378	 = Validation score   (f1_macro)
	7.81s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: RandomForestEntr_BAG_L1 ...
	No hyperparameter search space specified for RandomForestEntr_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForestEntr_BAG_L1 ...
	0.4521	 = Validation score   (f1_macro)
	13.94s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1

...

Fitted model: NeuralNetFastAI_BAG_L3/2fef927f ...
	0.5675	 = Validation score   (f1_macro)
	459.85s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: XGBoost_BAG_L3 ...


  0%|          | 0/1 [00:00<?, ?it/s]

	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.19%)
Fitted model: XGBoost_BAG_L3/T1 ...
	0.5789	 = Validation score   (f1_macro)
	216.29s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L3 ...
/usr/local/lib/python3.10/dist-packages/ray/air/config.py:803: UserWarning: Setting a `RunConfig.local_dir` is deprecated and will be removed in the future. If you are not using remote storage,set the `RunConfig.storage_path` instead. Otherwise, set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable to control the local cache location.
  warnings.warn(


+----------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch_BAG_L3   |
+----------------------------------------------------------+
| Search algorithm                 SearchGenerator         |
| Scheduler                        FIFOScheduler           |
| Number of trials                 1                       |
+----------------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20231209_114616/models/NeuralNetTorch_BAG_L3



Fitted model: NeuralNetTorch_BAG_L3/31648762 ...
	0.5519	 = Validation score   (f1_macro)
	549.61s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L3 ...
	Fitting 35 child models (S1F1 - S5F7) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.28%)
	0.5242	 = Validation score   (f1_macro)
	293.92s	 = Training   runtime
	0.64s	 = Validation runtime
Fitting model: WeightedEnsemble_L4 ...
	Ensemble Weights: {'CatBoost_BAG_L2/T1': 0.319, 'LightGBM_BAG_L2/T1': 0.181, 'CatBoost_BAG_L1/T1': 0.138, 'XGBoost_BAG_L1/T1': 0.074, 'LightGBMXT_BAG_L2/T1': 0.074, 'XGBoost_BAG_L2/T1': 0.053, 'LightGBM_BAG_L3/T1': 0.053, 'CatBoost_BAG_L3/T1': 0.043, 'LightGBMXT_BAG_L3/T1': 0.032, 'KNeighborsDist_BAG_L1': 0.011, 'NeuralNetFastAI_BAG_L1/f107639c': 0.011, 'NeuralNetFastAI_BAG_L3/2fef927f': 0.011}
	0.5465	 = Validation score   (f1_macro)
	13.4s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runt

In [32]:
pred_y = predictor.predict(test)
pred_y

0       1
1       0
2       1
3       1
4       0
       ..
9995    1
9996    1
9997    1
9998    0
9999    0
Name: target, Length: 10000, dtype: int64

In [33]:
sub['target'] = pred_y
sub.to_csv('auto_new_col3_167+L2+adverse.csv',index=False)